In [1]:
#Imports neccessary packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import econtools.metrics as mt
from prettytable import PrettyTable

#Reads *.dta files into pandas
table1 = pd.read_stata("C:\\Users\\littl\\OneDrive\\Desktop\\ReplicationsModern\\LegalOriginsAndFemaleHIV\\MS-AER-2015-1047-DATA\\TABLES\\hiv-table1.dta")
table2 = pd.read_stata("C:\\Users\\littl\\OneDrive\\Desktop\\ReplicationsModern\\LegalOriginsAndFemaleHIV\\MS-AER-2015-1047-DATA\\TABLES\\hiv-table2.dta")

In [2]:
%%capture --no-stdout

table1 = table1.dropna()
table2 = table2.dropna()

#Converts strings to numerics
table1.replace(('Yes', 'No'), (1, 0), inplace=True)
table2.replace(('Yes', 'No'), (1, 0), inplace=True)

#This starts the years at zero instead of i.e. 1980
#table1["yt"] = pd.to_numeric(table1['yt'],downcast="integer")
#table2["yt"] = pd.to_numeric(table2['yt'],downcast="integer")

#Factorizes qualitative data into quantitative values
table1["tribe_code"] = pd.factorize(table1["tribe_code"])[0]
table2["tribe_code"] = pd.factorize(table2["tribe_code"])[0]
table1["country"] = pd.factorize(table1["country"])[0]
table2["country"] = pd.factorize(table2["country"])[0]

#Creates dummy variables for tribe codes
itribe_code = pd.get_dummies(table1['tribe_code'],prefix="tribe_code",dtype=float)
for i in range(0,len(itribe_code.columns)):
    string = itribe_code.columns[i]
    table1[string] = itribe_code[string]
    
#Creates dummy variables for tribe codes
itribe_code = pd.get_dummies(table2['tribe_code'],prefix="tribe_code",dtype=float)
for i in range(0,len(itribe_code.columns)):
    string = itribe_code.columns[i]
    table2[string] = itribe_code[string]   

In [3]:
%%capture --no-stdout

tribedummy = []
tribedummy2 = []
for element in table1.columns:
    if element.startswith("tribe_code_"):
        tribedummy.append(element)
for element in table2.columns:
    if element.startswith("tribe_code_"):
        tribedummy2.append(element)

tribedummy.pop(0)
tribedummy2.pop(0)

# Add intercept to the independent variables
cons = []
for i in range(0, len(table1)):
    cons.append(1)
table1["const"] = cons
cons = []
for i in range(0, len(table2)):
    cons.append(1)
table2["const"] = cons

In [4]:
table1['combined_cluster'] = table1[['country', 'tribe_code']].astype(str).agg('_'.join, axis=1)


le200 = table1[table1['rdkm']<=200]
le150 = table1[table1['rdkm']<=150]
le100 = table1[table1['rdkm']<=100]
le100t1 = table1[(table1['rdkm']<=100) & (table1['target']==1.0)]
le100t0 = table1[(table1['rdkm']<=100) & (table1['target']==0.0)]
regression_names = [le200,le150,le100,le100t1,le100t0]
df_names = ["le200","le150","le100","le100t1","le100t0"]

In [5]:
#Sets the endogenous variable
y = ["hivpos"]
z=-1
coefficients = ["Coefficients:"]
se = ["Standard Errors:"]
nobs = ["Number of Observations:"]

for n in regression_names:
    z += 1
    x = ["commonlaw", "wifeage", "wifenoeduc", "gdp_pop_ppp2004", "abs_latitude", "longitude", "rain_min", "humid_max", "low_temp", "yt",  'j_pd0', 'j_l0708', 'j_km2split', 'j_mean_ele', 'j_mean_sui', 'j_malarias', 'j_petroleu',  'j_diamondd', 'j_capdista', 'j_seadist1', 'j_borderdi', 'southafrica', 'centralafrica', 'eastafrica', 'westafrica', 'rdkm', 'rdkmsq','const']
    x = x+tribedummy
    while 0 < 1:
        try:
            result = mt.reg(n, y, x, check_colinear=True)
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                break
    # Set up the independent and dependent variables
    X = n[x]
    Y = n["hivpos"]
    
    model = sm.OLS(Y,X,hasconst=True).fit()
    
    # Get the number of unique groups for the combined cluster variable
    num_clusters = n['combined_cluster'].nunique()
    num_obs = len(n)

    # Calculate the degrees of freedom adjustment, considering the combined clusters
    clusters_per_obs = num_clusters / num_obs
    df_correction = (1 / (1 - clusters_per_obs)) * ((num_obs - 1) / (num_obs - model.df_model - 1))
    
    # Get cluster-robust standard errors using the combined cluster variable
    robust_errors = model.get_robustcov_results(cov_type='cluster', groups=n['combined_cluster'], df_correction=df_correction)
    
    coefficients.append(round(model.params.commonlaw,3))
    se.append(round(robust_errors.bse[0],3))
    nobs.append(int(model.nobs))
    # print("The coeffecient of commonlaw in regression {} is {}".format(df_names[z],round(coefficients[z],3)))
    # print("The corresponding standard error is {}".format(round(se[z],3)))
    # print("The number of observations is {}".format(int(nobs[z])))

In [6]:
%%capture --no-stdout

table2['combined_cluster'] = table2[['country', 'tribe_code']].astype(str).agg('_'.join, axis=1)

le200_2 = table2[table2['rdkm']<=200]
le150_2 = table2[table2['rdkm']<=150]
le100_2 = table2[table2['rdkm']<=100]
le100t1_2 = table2[(table2['rdkm']<=100) & (table2['target']==1.0)]
le100t0_2 = table2[(table2['rdkm']<=100) & (table2['target']==0.0)]
regression_names_2 = [le200_2,le150_2,le100_2,le100t1_2,le100t0_2]
df_names_2 = ["le200_2","le150_2","le100_2","le100t1_2","le100t0_2"]

In [7]:
#Sets the endogenous variable
y = ["malehivpos"]
z=-1
coefficients_2 = ["Coefficients:"]
se_2 = ["Standard Errors:"]
nobs_2 = ["Number of Observations:"]

for n in regression_names_2:
    z += 1
    x = ['commonlaw', 'maleage', 'malenoeduc', 'gdp_pop_ppp2004', 'abs_latitude', 'longitude', 'rain_min', 'humid_max', 'low_temp', 'yt', 'j_pd0', 'j_l0708', 'j_km2split', 'j_mean_ele', 'j_mean_sui', 'j_malarias', 'j_petroleu', 'j_diamondd', 'j_capdista', 'j_seadist1', 'j_borderdi', 'rdkm', 'rdkmsq','const']
    x = x+tribedummy2
    while 0 < 1:
        try:
            result = mt.reg(n, y, x, check_colinear=True)
            break
        except ValueError as err:
            errstring = err.args[0]
            varlist = errstring.split("\n")
            varlist.pop(0)
            for i in range(0,len(varlist)):
                x.remove(varlist[i])
            if len(varlist) == 0:
                break
    # Set up the independent and dependent variables
    X = n[x]
    Y = n["malehivpos"]
    
    model = sm.OLS(Y,X,hasconst=True).fit()
    
    # Get the number of unique groups for the combined cluster variable
    num_clusters = n['combined_cluster'].nunique()
    num_obs = len(n)

    # Calculate the degrees of freedom adjustment, considering the combined clusters
    clusters_per_obs = num_clusters / num_obs
    df_correction = (1 / (1 - clusters_per_obs)) * ((num_obs - 1) / (num_obs - model.df_model - 1))
    
    # Get cluster-robust standard errors using the combined cluster variable
    robust_errors = model.get_robustcov_results(cov_type='cluster', groups=n['combined_cluster'], df_correction=df_correction)
    
    coefficients_2.append(round(model.params.commonlaw,3))
    se_2.append(round(robust_errors.bse[0],3))
    nobs_2.append(int(model.nobs))
    # print("The coeffecient of commonlaw in regression {} is {}".format(df_names[z],round(coefficients[z],3)))
    # print("The corresponding standard error is {}".format(round(se[z],3)))
    # print("The number of observations is {}".format(int(nobs[z])))

In [8]:
table = PrettyTable()

table2 = PrettyTable()

# Set table style
table.border = True
table.header = True
table.align = 'c'
table2.border = True
table2.header = True
table2.align = 'c'

table.field_names = ["Table 1"]+df_names
table2.field_names = ["Table 2"]+df_names_2

table.add_rows([coefficients,se,nobs])
table2.add_rows([coefficients_2,se_2,nobs_2])

print(table)
print(table2)

+-------------------------+--------+-------+-------+---------+---------+
|         Table 1         | le200  | le150 | le100 | le100t1 | le100t0 |
+-------------------------+--------+-------+-------+---------+---------+
|      Coefficients:      | 0.016  | 0.018 | 0.019 |  0.016  |  0.008  |
|     Standard Errors:    | 0.005  | 0.005 | 0.005 |  0.005  |  0.009  |
| Number of Observations: | 118903 | 99511 | 77336 |  55507  |  21829  |
+-------------------------+--------+-------+-------+---------+---------+
+-------------------------+---------+---------+---------+-----------+-----------+
|         Table 2         | le200_2 | le150_2 | le100_2 | le100t1_2 | le100t0_2 |
+-------------------------+---------+---------+---------+-----------+-----------+
|      Coefficients:      |  0.001  |  0.001  |  -0.001 |   -0.003  |   0.002   |
|     Standard Errors:    |  0.004  |  0.004  |  0.004  |   0.004   |   0.009   |
| Number of Observations: |  50754  |  40780  |  31189  |   24261   |    6928  